In [1]:
from ipywidgets import interact
%matplotlib notebook
%display latex

In [12]:
def mu(v0,v1,*_):
    """
    Returns Mumford slope for Chern character given by arguments
    
    Parameters:
    - ch_0
    - ch_1
    ... [can include further terms which will be ignored]
    
    Usage example:
    >>> mu(1,2,1)
    2
    """
    return v1/v0

def Q_tilt(v0,v1,v2,*_):
    """
    Returns the bogomolov form $Q^{\\mathrm{tilt}}$
    evaluated on given Chern character
        
    Parameters:
    - ch_0
    - ch_1
    - ch_2
    ... [can include further terms which will be ignored]
    
    Usage example:
    >>> Q_tilt(1,2,1)
    2
    """
    return v1**2 - 2*v0*v2


In [3]:
var("v0 v1 v2 v3")
v = v0,v1,v2,v3
var("s alpha beta")

Theta = lambda *v: \
        lambda alpha, beta, s: \
         (beta - mu(*v))**2 - alpha**2 - Q_tilt(*v)

def Theta_minus_param(t,*v):
    Q = Q_tilt(*v)
    beta = mu(*v) - cosh(t)*sqrt(Q)
    alpha = sinh(t)*sqrt(Q)
    return alpha,beta

def Theta_plus_param(t,*v):
    Q = Q_tilt(*v)
    beta = mu(*v) + cosh(t)*sqrt(Q)
    alpha = sinh(t)*sqrt(Q)
    return alpha,beta


Gamma = lambda v0,v1,v2,v3: \
        lambda alpha, beta, s: \
         v0*beta**3 - 3*v1*beta**2 + (6*v2-(6*s+1)*v0*alpha**2)*beta \
         +((6*s+1)*v1*alpha**2 - 6*v3)

# Example with $v=(1,0,-1,1)$

In [4]:
stab = alpha, beta, s
v = 1,0,-1,1

Expression for $\Theta_v$ curve:

In [5]:
Theta(*v)(*stab)

-alpha^2 + beta^2 - 2

Expression for $\Gamma_v$ curve:

In [6]:
Gamma(*v)(*stab)

beta^3 - (alpha^2*(6*s + 1) + 6)*beta - 6

Parametrization for $\Theta^{-}_v$:

In [7]:
var("t")
Theta_minus_param(t,*v)

(sqrt(2)*sinh(t), -sqrt(2)*cosh(t))

In [51]:
def plot_func(v0=1, v1=0, v2=-1, v3=1, plot_range=5):
    v = v0,v1,v2,v3
    var("alpha beta")
    s=1/3
    print(f"Q tilt on v={v}: {Q_tilt(*v)}")
    stab = alpha, beta, s
    mu_v = mu(*v)
    
    
    
    
    return (
        # Gamma curve
        implicit_plot(
          Gamma(*v)(*stab),
          (beta, mu_v - plot_range, mu_v + plot_range),
          (alpha, 0, plot_range),
          legend_label="Gamma",
          rgbcolor="green"
        )
        # beta = mu vertical line
        + line([(mu_v,0),(mu_v,plot_range)], rgbcolor=(1, 0, 0), legend_label="$\\beta=\\mu$")
        # Theta curve
        + implicit_plot(
          Theta(*v)(*stab),
          (beta, mu_v-plot_range, mu_v+plot_range),
          (alpha, 0, plot_range)
        )
        # Intersection point of the Gamma and Theta curves
#         + sum((
#             text(f"{(beta,alpha)}",(beta,alpha)) for alpha, beta in
#             (
#                 (alpha, beta) for alpha, beta in
#                 map(
#                     # Unpack list of symbolic expressions like [alpha==1,beta==2]
#                     # to just a tuple of values like (1,2)
#                     lambda res: (res[0].rhs(), res[1].rhs()),
#                     # All intersection points of the Gamma an Theta curve
#                     solve([Gamma(*v)(*stab)==0,Theta(*v)(*stab)==0],alpha,beta)
#                 )
#                 # Filter (alpha,beta) intersection points which are actually on
#                 # the plot
#                 if alpha.imag()==0 and beta.imag()==0 and alpha>=0
#             )
#         ))
    )

interact(plot_func)

interactive(children=(IntSlider(value=1, description='v0', max=3, min=-1), IntSlider(value=0, description='v1'…

<function plot_func at 0x6ffe76ed8290>

In [50]:
v = 1,0,-1,1
var("alpha beta")
stab = alpha, beta, 1/3

for alpha, beta in filter(
    lambda stab: stab[0].imag()==0 and stab[1].imag()==0 and stab[0]>=0,
    map(
        lambda aoeu: (aoeu[0].rhs(), aoeu[1].rhs()),
        solve([Gamma(*v)(*stab)==0,Theta(*v)(*stab)==0],alpha,beta)
    )
):
    print(alpha,beta)

0.2829908393166625 -1.442249593306995


In [68]:
v = 1,0,-1,1
var("alpha beta t")
s = 1/3
var("A")
stab = alpha, beta, s
expression = Gamma(*v)(*Theta_minus_param(t,*v),s)
pretty_print(expression)
pretty_print(expression.subs(exp(t) = A))
plot(expression,t).show()
print("helo")

SyntaxError: keyword can't be an expression (<ipython-input-68-3098150f123d>, line 8)

In [59]:
res = solve(expression,t)
res

[t == arccosh(-1/2*I*sqrt(3)*(1/2)^(1/3)*(-sqrt(2))^(1/3) + I*sqrt(3)*(1/2)^(2/3)/(-sqrt(2))^(1/3) - 1/2*(1/2)^(1/3)*(-sqrt(2))^(1/3) - (1/2)^(2/3)/(-sqrt(2))^(1/3)), t == arccosh(1/2*I*sqrt(3)*(1/2)^(1/3)*(-sqrt(2))^(1/3) - I*sqrt(3)*(1/2)^(2/3)/(-sqrt(2))^(1/3) - 1/2*(1/2)^(1/3)*(-sqrt(2))^(1/3) - (1/2)^(2/3)/(-sqrt(2))^(1/3)), t == arccosh((1/2)^(1/3)*(-sqrt(2))^(1/3) + 2*(1/2)^(2/3)/(-sqrt(2))^(1/3))]

In [63]:
simplify(res[1].rhs().imag())

arctan2(0, -sqrt(-1/4*I*sqrt(3)*2^(5/6)*(-1)^(1/3) - 1/2*I*sqrt(3)*2^(1/6)*(-1)^(2/3) + 1/4*2^(5/6)*(-1)^(1/3) - 1/2*2^(1/6)*(-1)^(2/3) + 1)*sqrt(-1/4*I*sqrt(3)*2^(5/6)*(-1)^(1/3) - 1/2*I*sqrt(3)*2^(1/6)*(-1)^(2/3) + 1/4*2^(5/6)*(-1)^(1/3) - 1/2*2^(1/6)*(-1)^(2/3) - 1) - 1/2*2^(5/6) - 2^(1/6))